<a href="https://colab.research.google.com/github/tejaspradhan/Graph-Neural-Networks/blob/main/pytorch-geometric/Tutorial_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!python -c "import torch; print(torch.__version__)"
!python -c "import torch; print(torch.version.cuda)"
!python --version

1.10.0+cu111
11.1
Python 3.7.12


In [ ]:
! pip install --no-index torch-scatter -f https://pytorch-geometric.com/whl/torch-1.10.0+cpu.html
! pip install --no-index torch-sparse -f https://pytorch-geometric.com/whl/torch-1.10.0+cpu.html
! pip install --no-index torch-cluster -f https://pytorch-geometric.com/whl/torch-1.10.0+cpu.html
! pip install --no-index torch-spline-conv -f https://pytorch-geometric.com/whl/torch-1.10.0+cpu.html
! pip install torch-geometric

In [24]:
import torch_geometric
from torch_geometric.datasets import Planetoid
import torch
import torch.nn.functional as F
from torch_geometric.nn import SAGEConv


In [25]:
dataset = Planetoid(root='tutorial1' ,name='cora')

In [26]:
dataset.data

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])

In [27]:
dataset.num_node_features
# there are 2708 nodes of 1433 node features each 

1433

In [28]:
data = dataset[0]

In [29]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.conv = SAGEConv(dataset.num_features,
                             dataset.num_classes,
                             aggr="max") # max, mean, add ...)

    def forward(self):
        x = self.conv(data.x, data.edge_index)
        return F.log_softmax(x, dim=1)

In [30]:
device = torch.device('cuda' if torch.cuda.is_available() and use_cuda_if_available else 'cpu')
model, data = Net().to(device), data.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

In [31]:
device

device(type='cpu')

In [33]:
def train():
    model.train()
    optimizer.zero_grad()
    F.nll_loss(model()[data.train_mask], data.y[data.train_mask]).backward()
    optimizer.step()


def test():
    model.eval()
    logits, accs = model(), []
    for _, mask in data('train_mask', 'val_mask', 'test_mask'):
        pred = logits[mask].max(1)[1]
        acc = pred.eq(data.y[mask]).sum().item() / mask.sum().item()
        accs.append(acc)
    return accs

Epoch: 010, Val: 0.7220, Test: 0.7150
Epoch: 020, Val: 0.7220, Test: 0.7150
Epoch: 030, Val: 0.7220, Test: 0.7150
Epoch: 040, Val: 0.7220, Test: 0.7150
Epoch: 050, Val: 0.7240, Test: 0.7000
Epoch: 060, Val: 0.7320, Test: 0.7060
Epoch: 070, Val: 0.7320, Test: 0.7060
Epoch: 080, Val: 0.7320, Test: 0.7060
Epoch: 090, Val: 0.7320, Test: 0.7060
